In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats as ss
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.filterwarnings(action='once')
%matplotlib inline

In [2]:
data = pd.read_csv("CleanedData.csv", header=[0,1,2,3], index_col=0)

In [3]:
get_slice = lambda name, level: data.loc[:,data.columns.get_level_values(level)==name].droplevel(level=[1,2,3],axis=1)

In [ ]:
#Calculates the Cramer's V for 2 named columns in the survey dataset
#and is based off the code block from the article in the 2nd link above.

def cramers_v(xname, yname):
    x = np.array(data.loc[:,data.columns.get_level_values(0)==xname]).ravel()
    y = np.array(data.loc[:,data.columns.get_level_values(0)==yname]).ravel()
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.round(np.sqrt(phi2corr/min((kcorr-1),(rcorr-1))),2)

In [ ]:
def chi2_test(xname, yname):
    x = np.array(data.loc[:,data.columns.get_level_values(0)==xname]).ravel()
    y = np.array(data.loc[:,data.columns.get_level_values(0)==yname]).ravel()
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)
    val, p = chi2[0], chi2[1]
    
    return np.round(p,4)*100

In [ ]:
def get_mat(func, mat_slice,plot,title="",names=[], show_sig=False):
    corr = mat_slice.corr()
    
    for r in mat_slice.columns:
        for c in mat_slice.columns:
            corr.loc[r,c] = func(r,c)
            
    if plot==False: return corr
    
    fsize = max(5,corr.shape[0]-6)
    plt.figure(figsize = (fsize,fsize))

    ax = sns.heatmap(corr, cmap="hot", annot=True, xticklabels=names, yticklabels=names)
    plt.title(title)
    plt.xlabel("")
    plt.ylabel("")
    plt.show()

In [ ]:
slice_example = get_slice("PolIden",3).loc[:,"selfdescr_ccap_1_baseline":"selfdescr_ccap_99_baseline"]
translations = ["Libertarian", "Socialist", "Green", "Environmentalist", "Liberal", "Moderate", "Conservative",\
                         "Radical", "Progressive", "Traditional", "Christian", "Feminist", "Fundamentalist", "None"]

In [ ]:
get_mat(cramers_v, slice_example, plot=True, title="Association (Cramer's V)", names=translations)

In [ ]:
get_mat(chi2_test, slice_example, plot=True, title="Test of Indenpendence P-value %", names=translations)

In [ ]:
slice2 = get_slice("Pol_Approval",3).loc[:,"fav_trump_2018":"fav_mcconnell_2018"]

In [ ]:
print(slice2.columns)